In [ ]:
# https://sites.google.com/site/ryunosukehm/study/ml-with-python/impact-ss2017/hmm_toy_example
# https://www.kabuku.co.jp/developers/hmm
# http://keik-117.hatenablog.com/entry/2016/07/05/213903
import numpy as np
from hmmlearn import hmm

# https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn-hmm
# 隠れた状態は指番号 観測できるのは音符列
components = (5 * 5) * 2 # (1->1, 1->2, ..., 5->4, 5->5) * (上行, 下行)
model = hmm.GaussianHMM(n_components=components, covariance_type="diag")

model.startprob_ = np.array(np.ones(components)) / components

# ある指番号から指番号への遷移確率
# t = np.array([
#     np.kron([1, 0, 0, 0, 0] * 2, np.ones(5)),
#     np.kron([0, 1, 0, 0, 0] * 2, np.ones(5)),
#     np.kron([0, 0, 1, 0, 0] * 2, np.ones(5)),
#     np.kron([0, 0, 0, 1, 0] * 2, np.ones(5)),
#     np.kron([0, 0, 0, 0, 1] * 2, np.ones(5))
# ]) / (5 * 2)
t = np.array([
    np.array(([0]*0) +[0.00, 0.25, 0.25, 0.25, 0.25]+([0]*20) + ([0]*0)+[0.00, 0.50, 0.30, 0.10, 0.10] +([0]*20)) / 2,
    np.array(([0]*5) +[0.10, 0.00, 0.30, 0.30, 0.30]+([0]*15) + ([0]*5)+[0.85, 0.00, 0.05, 0.05, 0.05] +([0]*15)) / 2,
    np.array(([0]*10)+[0.10, 0.05, 0.00, 0.50, 0.35]+([0]*10) + ([0]*10)+[0.40, 0.50, 0.00, 0.05, 0.05]+([0]*10)) / 2,
    np.array(([0]*15)+[0.10, 0.05, 0.05, 0.00, 0.80]+([0]*5)  + ([0]*15)+[0.25, 0.25, 0.45, 0.00, 0.05]+([0]*5)) / 2,
    np.array(([0]*20)+[0.20, 0.20, 0.20, 0.20, 0.20]+([0]*0)  + ([0]*20)+[0.20, 0.15, 0.15, 0.50, 0.00]+([0]*0)) / 2,
])
model.transmat_ = np.tile(t, (5 * 2, 1))

In [ ]:
# ノードの出力
# ５指からの上行のみ特別扱い
model.means_ = np.array([
    # 0 上行 1指 -> 
    [0],
    [1],
    [2],
    [3],
    [4],
    # 上行 2指 -> 
    [1],
    [0],
    [1],
    [2],
    [3],
    # 上行 3指 -> 
    [1],
    [1],
    [0],
    [1],
    [2],
    # 上行 4指 -> 
    [1],
    [1],
    [1],
    [0],
    [1],
    # 上行 5指 -> 
    [5],
    [5],
    [5],
    [5],
    [5],
    # 25 下行 1指     -> 
    [0],
    [-1],
    [-1],
    [-1],
    [-1],
    
    [-1],
    [0],
    [-1],
    [-1],
    [-1],
    
    [-2],
    [-1],
    [0],
    [-1],
    [-1],
    
    [-3],
    [-2],
    [-1],
    [0],
    [-1],

    [-4],
    [-3],
    [-2],
    [-1],
    [0],
])
sigma = 1
covars = np.ones((components, 1)) * sigma
model.covars_ = covars

In [ ]:
X, Z = model.sample(10)
Z % 5 + 1

In [ ]:
import music21
def octave_dist(midi):
    d = [0, 0.5, 1, 1.5, 2, 3, 3.5, 4, 4.5, 5, 5.5, 6, 7]
    return midi // 12 * d[-1] + d[midi % 12]

def to_inputs(notes):
    midies = [note.pitch.midi for note in notes]
    # differs = [midies[i + 1] - midies[i] for i in range(len(midies) - 1)]
    outputs = [0] * len(midies)
    for i in range(len(midies) - 1):
        o1 = octave_dist(midies[i + 1])
        o = octave_dist(midies[i])
        outputs[i + 1] = o1 - o
    return outputs

In [ ]:
# 入力
stream = music21.converter.parse('./mxl/note_example1.xml')
notes = stream.flat.notes
outputs = to_inputs(notes)
# 予測
fingers = model.predict(np.array(outputs).reshape(len(outputs), 1)) % 5 + 1
# 楽譜化
s = music21.stream.Stream()
for n in notes:
    s.append(music21.note.Note(n.nameWithOctave, quarterLength=n.quarterLength))
for i in range(len(notes)):
    s[i].articulations.append(music21.articulations.Fingering(fingers[i]))
s.show()

In [ ]:
# 入力
stream = music21.converter.parse('./mxl/note_example2.xml')
notes = stream.flat.notes
outputs = to_inputs(notes)
# 予測
fingers = model.predict(np.array(outputs).reshape(len(outputs), 1)) % 5 + 1
# 楽譜化
s = music21.stream.Stream()
for n in notes:
    s.append(music21.note.Note(n.nameWithOctave, quarterLength=n.quarterLength))
for i in range(len(notes)):
    s[i].articulations.append(music21.articulations.Fingering(fingers[i]))
s.show()